# Cash Conversion: Measuring a Company's Effectiveness

"Cash is King" is a common saying in most businesses; cash is the most liquid asset, enabling business owners to immediately exchange cash for goods and services. However, how can you tell if your business is a *Cash King*? While there are many different metrics a business owner can look at, the *Cash Conversion Cycle* is a well-known measurement for gauging a business's capability to generate cash.

## Understanding Cash Conversion Cycle

In essence, the *Cash Conversion Cycle* measures the amount of time a business sees a return on their investment. For example, a Company must invest funds in order to build and sell a product. The following scenario illustrates a typical workflow in which we can derive the *Cash Conversion Cycle*:

The aforementioned criteria can easily be explained in the following, typical scenario:
1. Company A purchase materials from Supplier B to build a product and receives an invoice (see: "`payable`")
2. Company A manufacturers and warehouses the product until the product is sold (see: "`inventory`")
3. Company A sells the product to Customer B and sends the customer an invoice (see: "`sales`")

As you can see, along each step of the way, there is a tangible cost in terms of money and time; the longer a customer takes to pay in Step #3, the longer your return on investment will be!

The aforementioned steps can be formally broken down into three criteria:
- **Days Payable Outstanding (DPO)**
    - How long does it take *Company A* to pay its suppliers?
    - DPO = payables/(cogs/days)
- **Days Inventory Outstanding (DIO)**
    - How long will the product stay in a warehouse before it's sold?
    - DIO = (inventory / cogs)*days
- **Days Sales Oustanding (DSO)**
    - How quickly does *Company A*'s customers pay?
    - DSO = receivables/(revenue/days)

Combining all of these criteria will yield the **Cash Conversion Cycle (CCC)**, measured in "days":
    
`CCC = DIO + DSO - DPO`

**In general, a lower Cash Conversion Cycle means the company is more effective at building and selling a product. **


In [2]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt



## Analyzing Microsoft's Cash Conversion Cycle

In order to put our theory into practice, let's analyze Microsoft's Cash Conversion Cycle. In order to do so, we'll need access to Micrsoft's Income Statement and Balance Sheet. Fortunately, the Security Exchange Commission has an open API that we can access in order to pull Microsoft's Income Statement and Balance Sheet figures. 


In [3]:
#Fetch Financial information for 3 years from SEC
json = pd.read_json("http://edgaronline.api.mashery.com/v2/corefinancials/ann.json?primarysymbols=MSFT&numperiods=3&appkey=[API KEY HERE]")

#Nested JSON data needs to be extracted
data2016 = pd.DataFrame(json['result'][0][0]['values'])
data2015 = pd.DataFrame(json['result'][0][1]['values'])
data2014 = pd.DataFrame(json['result'][0][2]['values'])

HTTPError: HTTP Error 403: Forbidden

## Munging through Nested JSON

EDGAR's API is formatted in .json which is supposedly easy to parse. However, I had a difficult time parsing through the nested JSON that EDGAR was presenting. Apparently, parsing through nested JSON is a common problem for most `pandas` users. 

In order to make the data easier to manipulate, I merged each year's financial data in a dataframe.

In [287]:
#Merge 3-year financial data into a workable dataframe
df1 = pd.merge(data2014, data2015, on='field')
df2 = pd.merge(df1,data2016,on='field')

#Merge multiple dataframes together (room for improvement)
df2.columns = ['Field','2014','2015','2016']
df3 = df2.set_index("Field")

df3.head(5)

,2014,2015,2016
Field,,,
cik,0000789019,0000789019,0000789019
companyname,MICROSOFT CORP,MICROSOFT CORP,MICROSOFT CORP
entityid,8528,8528,8528
primaryexchange,Nasdaq Global Market,Nasdaq Global Market,Nasdaq Global Market
marketoperator,NASDAQ,NASDAQ,NASDAQ


## Extracting relevant information

As we've previously discussed, we are looking specifically at the Income Statement and Balance Sheet for important information. In order to calculate the DSO, DPO, and DIO, we'll need the following information:

- Revenue
- Cost of Revenue
- Total Receivables
- Inventories
- Payables

Unfortunately, during my data extraction, I could not find `payables` in the EDGAR API. Therefore, as you can see below, I manually entered the information. Note, I obtained the payable information from Microsoft's [10-K](https://en.wikipedia.org/wiki/Form_10-K) (annual) reports. 

In [252]:
#Extract relevant metrics from raw, merged dataframe

#global
days = 365

#Income Statement
revenue = df3.loc['totalrevenue',:]
cogs = df3.loc['costofrevenue',:]

#Balance Sheet - Assets
receivables = df3.loc['totalreceivablesnet',:]
inventory = df3.loc['inventoriesnet',:]

#Balance Sheet - Liabilities
payables = [7432000000,6591000000,6898000000] #hardcoded since EDGAR data was missing

# Calculating Days Sales Outstanding

Calculating DSO includes dividing `receivables` by our `revenue` per `day`. Abstractly, one can view this metric as the ratio between the amount of cash that needs to be collected and your average daily sales; if a company has a high DSO, they are making sales but not collecting the cash effectively.

In [282]:
#Days Sales Outstanding
dso = receivables/(revenue/days)
dso

2014    82.1526
2015    69.8485
2016    78.1892
dtype: object

It appears that Microsoft has reduced their average DSO since 2014. Reduction in DSOs can be attributed to more favorable contracting terms and/or providing incentives to their customers to pay earlier (e.g a "10/2" schedule gives customers a 2% discount on the invoice if they pay it within 10 days). 

# Calculating Days Inventory Outstanding

Days Inventory Outstanding will showcase the amount of time inventory stays within a warehouse. Ideally the DIO would be zero, meaning that once a product is manufacturered, it is immediately sold. If a business were able to carry no inventory, there would be no need for renting a warehouse, thus freeing up capital that can be spent on other facets of the business. However, in reality, most businesses do carry some type of inventory; for manufacturers the inventory is a finished product.

*NOTE: Other industries can use the idea of "inventory" to increase efficiency. For example, hotels might consider vacant rooms as "inventory" since they're sitting idle & unsold.*

In [284]:
#Days Inventory
dio = (inventory / cogs)*days
dio


2014    35.8557
2015     32.061
2016    25.0645
dtype: object

Here we can see that Microsoft has consistently improved its DIO metric. An explanation for improved DIO could be due to a better Sales organization (i.e selling products faster), or improved production/inventory operations (balancing production and inventory with customer demand). 

*NOTE: Inventory Operations is a fascinating and well-researched issue. I would highly recommend reading Eliyahu Goldratt's ["The Goal"](https://www.amazon.com/dp/B002LHRM2O/ref=dp-kindle-redirect?_encoding=UTF8&btkr=1) if you're interested in this topic. Eliyahu describes the [Theory of Constraints](https://en.wikipedia.org/wiki/Theory_of_constraints) and it's financial impact on large and small businesses.



## Calculating Days Payable Outstanding

Days Payable Outstanding is used to measure the amount of time a Company takes to pay its suppliers. DPO is a metric that can be easily controlled. Ideally, the DPO value would be infinite (meaning that you never pay your suppliers). However, in practice, if you failed to pay your suppliers, your company would be sued and gain a poor reputation.

In [253]:
#Days Payable
dpo = payables/(cogs/days)
dpo

2014     100.18
2015    72.8166
2016    76.8081
Name: costofrevenue, dtype: object

Clearly, we can see the Microsoft was paying vendors an average of 100 days after receiving an invoice. However, in 2015 and 2016, Microsoft greatly reduced their DPO to a respectable 72-76 days - perhaps Microsoft was able to receive more favorable terms from their suppliers if they agreed to pay earlier.

## Finally, Cash Conversion Cycle

With DIO, DSO, and DPO calculated we are ready to calculate the Cash Conversion Cycle.

In [229]:
#Cash Conversion Cycle
ccc = dio + dso - dpo
ccc

2014    17.8281
2015    29.0928
2016    26.4456
dtype: object

Remember, a lower CCC illustrates a more effective organization. Therefore, it appears that, while Microsoft has been more effective in generating cash since 2015, Microsoft was *most* effective in 2014. 

If we look back at our three criteria (DIO, DSO, and DPO), we can see that 2014's change in DPO had a huge impact! Since Microsoft was willing (or forced) to reduce their DPO, they negatively impacted their Cash Conversion Cycle. 

## Conclusion

While the aforementioned example was a simplistic view, the *Cash Conversion Cycle* of a company can highlight the (in)effectiveness of an organization. 

Ideally, with further investigation, I would have liked to compare Microsoft's CCC with industry competitors' to analyze Microsoft's relative performance. However, with limited time, I only focused on Microsoft's capabilities (none the less, still a valuable investigation).

Therefore, next time someone says "Cash is King", ask them how many days their Cash Conversion Cycle is :) 